# Test one; finding normal correlation: crime, neighbourhood statistics.

In [463]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from sklearn.linear_model import LinearRegression


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

I'm going to merge some of my datasets into one.

In [464]:
# RAW		CLEANED		USED
# 10		9			6

# missing: workplaces don't have neighbourhood, only street
# livibility: not usable in this state

#LABELS:
# RAW		CLEANED		USED
# 2			2			1


# [X] breda_population
# [X] education_levels
# [X] labor_force
# [!] livability
# [!] migration
# [X] public_nuisance_criminal
# [X] public_nuisance_noncriminal
# [X] police_response_times
# [X] weather_metrics
# [!] workplaces_info

# [X] crime_metrics_simplified
# [ ] crime_metrics


#time series
df_nuisance_criminal = pd.read_csv('../Datasets/cleaned/police_nuisance_criminal.csv')
df_nuisance_noncriminal = pd.read_csv('../Datasets/cleaned/police_nuisance_noncriminal.csv')
df_response_times = pd.read_csv('../Datasets/cleaned/police_response_times_breda.csv')

df_livability = pd.read_csv('../Datasets/cleaned/livability_metrics.csv')

#cross-sectional
df_education = pd.read_csv('../Datasets/cleaned/education_levels_breda_2021.csv')
df_labor = pd.read_csv('../Datasets/cleaned/labor_force_breda_2021.csv')
df_population = pd.read_csv('../Datasets/cleaned/breda_population_2020.csv')

#misc
df_weather = pd.read_csv('../Datasets/cleaned/weather_metrics.csv')


#label
df_label = pd.read_csv('../Datasets/cleaned/crime_metrics_simplified.csv')



# merging a little bit

df_merged_time_series = pd.merge(df_nuisance_criminal, df_nuisance_noncriminal, on=['Year', 'Month', 'NeighbourhoodCode'], how='outer')

#print("being:")
#print("nuisance reports:")
#print(df_merged_time_series.head())
#
#print("livability")
#print(df_livability.head())
#
#print("merging?")
##livability data is very fucked for somer reason??? I'll merge by hand  -- apparently if I try to merge with livability it creates duplicate rows for no reason at all
#cols_to_use = df_livability.columns.difference(df_merged_time_series.columns)
#print("dif:")
#print(cols_to_use)
#print("lets try again")
#cols_to_use = ['lbm','afw','fys','onv','soc','vrz','won','area']
#print("lets try merge now:")
##df_merged_time_series = df_merged_time_series.merge(df_livability, left_on=['Year', 'Month', 'NeighbourhoodCode'], right_on=['Year', 'Month', 'NeighbourhoodCode'])
##df_merged_time_series = pd.merge(df_merged_time_series, df_livability[cols_to_use], left_index=True, right_index=True, how='outer')
#df_merged_time_series = pd.merge(df_merged_time_series, df_livability, on=['Year', 'Month', 'NeighbourhoodCode'], how='outer')
#
#print(df_merged_time_series.head())
#
#df_merged_time_series.drop_duplicates()
#
#print(df_merged_time_series.head())
#df_merged_time_series


In [465]:
#weather

print(df_merged_time_series.head())
print("we:", df_weather.head())
df_merged_time_series = pd.merge(df_weather, df_merged_time_series, on="Month")
print("fin:", df_merged_time_series.head(), "end")
df_merged_time_series

  NeighbourhoodCode  Year  Month  NuisanceReportsCriminal  \
0        Valkenberg  2012      1                        0   
1        Valkenberg  2012      2                        0   
2        Valkenberg  2012      3                        1   
3        Valkenberg  2012      4                        1   
4        Valkenberg  2012      5                       28   

   NuisanceReportsNonCriminal  
0                           0  
1                           2  
2                           2  
3                           3  
4                           5  
we:    Month  DayC  NightC  Rain Days  Rainfall(mm)  Daylight hours
0      1     5       2          7         24.70            8.50
1      2     6       1          6         19.20           10.00
2      3    11       3          6         18.80           12.00
3      4    14       6          4         14.40           14.00
4      5    17       9          7         22.80           15.50
fin:    Month  DayC  NightC  Rain Days  Rainfall(mm) 

,Month,DayC,NightC,Rain Days,Rainfall(mm),Daylight hours,NeighbourhoodCode,Year,NuisanceReportsCriminal,NuisanceReportsNonCriminal
0,1,5,2,7,24.70,8.50,Valkenberg,2012,0,0
1,1,5,2,7,24.70,8.50,Valkenberg,2013,4,0
2,1,5,2,7,24.70,8.50,Valkenberg,2014,3,0
3,1,5,2,7,24.70,8.50,Valkenberg,2015,2,1
4,1,5,2,7,24.70,8.50,Valkenberg,2016,1,2
...,...,...,...,...,...,...,...,...,...,...
7387,12,6,3,8,29.20,8.00,Buitengebied Teteringen,2018,0,0
7388,12,6,3,8,29.20,8.00,Buitengebied Teteringen,2019,0,0
7389,12,6,3,8,29.20,8.00,Buitengebied Teteringen,2020,0,0
7390,12,6,3,8,29.20,8.00,Buitengebied Teteringen,2021,0,0


Imputating some missing data (specifically, from 2012 to 2015)

In [466]:
def addMissingYearRows(df):
	years_range = range(2012, 2016)
	months_range = range(1, 13)
	all_combinations = [(year, month) for year in years_range for month in months_range]

	new_rows = pd.DataFrame(all_combinations, columns=["Year", "Month"])
	merged_df = new_rows.merge(df, on=["Year", "Month"], how="left")
	return merged_df

#based on label
def addMissingNeighbourhoods(df):
	unique_neighborhoods = df_label['NeighbourhoodCode'].unique()
	missing_df = pd.DataFrame({'NeighbourhoodCode': unique_neighborhoods})
	return pd.merge(df, missing_df, on='NeighbourhoodCode', how='outer')



In [467]:

imputer = SimpleImputer(strategy='mean')

merged_df = addMissingYearRows(df_response_times)
numerical_cols = ["ResponseTimeScore", "SlowResponseTimePenalty"]
imputer = SimpleImputer(strategy="mean")

merged_df[numerical_cols] = imputer.fit_transform(merged_df[numerical_cols])
merged_df.sort_values(["Year", "Month"], inplace=True)
merged_df.reset_index(drop=True, inplace=True)

merged_df = merged_df.astype(int)


We need to calculate some of the missing numbers from the labor participation and education data.

In [468]:
df_merged_responses = pd.concat([merged_df, df_response_times])
#we merge the reponse times (Year, Month) and the time series (Year, Month, NeighbourhoodCode)
df_merged_timebased = pd.merge(df_merged_responses, df_merged_time_series, on=['Year', 'Month'], how='right')


In [469]:
df_labor = addMissingNeighbourhoods(df_labor)

numerical_cols = ["NetLaborParticipation","FlexibleContracts","SelfContract"]
df_labor[numerical_cols] = imputer.fit_transform(df_labor[numerical_cols])
df_labor.reset_index(drop=True, inplace=True)

df_labor[numerical_cols] = df_labor[numerical_cols].astype(int)

##and education
df_education = addMissingNeighbourhoods(df_education)
numerical_cols = ["PopulationEduLow","PopulationEduMedium","PopulationEduHigh"]

df_education[numerical_cols] = imputer.fit_transform(df_education[numerical_cols])
df_education.reset_index(drop=True, inplace=True)

df_education[numerical_cols] = df_education[numerical_cols].astype(int)


In [470]:
#Let's add relevant data to neighbourhoods
df_merged_neighbourhoods = pd.merge(df_labor, df_education, on=['NeighbourhoodCode'], how='left')

df_merged_neighbourhoods = pd.merge(df_merged_neighbourhoods, df_population, on=['NeighbourhoodCode'], how="left")

#Combine the two
df_final = pd.merge(df_merged_timebased, df_merged_neighbourhoods, on=['NeighbourhoodCode'], how='outer')

col_nbhc = df_final.pop("NeighbourhoodCode")
df_final.insert(0, "NeighbourhoodCode", col_nbhc)
df_final.sort_values(["NeighbourhoodCode", "Year", "Month"], inplace=True)

In [471]:
#to file
df_final.to_csv('../Datasets/finalized/merged_nuisance_response_education_employment_population.csv', index=False)

OK now the real deal.
We will create a very basic AI model to see if we can predict crimes at least a little bit.

In [472]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

from scipy.stats import spearmanr

In [473]:
# Set 'NeighbourhoodCode' as the index for both dataframes
df_final.set_index('NeighbourhoodCode', inplace=True)
df_label.set_index('NeighbourhoodCode', inplace=True)

# Merge data and label dataframes
merged_df = pd.merge(df_final, df_label, on=['Year', 'Month'])

In [474]:
#We can try to do this:
if True:
    print(merged_df.head())
    merged_df['CrimeRate'] = (merged_df["CrimeCount"] / merged_df["Inhabitants"]) * 100000
    merged_df.drop(['CrimeCount'], axis=1, inplace=True)
    print(merged_df.head())

merged_df

   Year  Month  ResponseTimeScore  SlowResponseTimePenalty  DayC  NightC  \
0  2012      1               0.00                     1.00     5       2   
1  2012      1               0.00                     1.00     5       2   
2  2012      1               0.00                     1.00     5       2   
3  2012      1               0.00                     1.00     5       2   
4  2012      1               0.00                     1.00     5       2   

   Rain Days  Rainfall(mm)  Daylight hours  NuisanceReportsCriminal  ...  \
0          7         24.70            8.50                        0  ...   
1          7         24.70            8.50                        0  ...   
2          7         24.70            8.50                        0  ...   
3          7         24.70            8.50                        0  ...   
4          7         24.70            8.50                        0  ...   

   Populationdensitykm2  AverageWOZ-valueofhouses(x1000euro)  \
0                  310

,Year,Month,ResponseTimeScore,SlowResponseTimePenalty,DayC,NightC,Rain Days,Rainfall(mm),Daylight hours,NuisanceReportsCriminal,...,Populationdensitykm2,AverageWOZ-valueofhouses(x1000euro),Percentuninhabited(%),Rentalproperies(%),Tradeandcatering%,Culture/recreationproperies%,Carsperhousehold,UrbanityLevel,Inhabitants,CrimeRate
0,2012,1,0.00,1.00,5,2,7,24.70,8.50,0,...,3109,442,2,27,0.14,0.09,1.30,0.25,5445,606.06
1,2012,1,0.00,1.00,5,2,7,24.70,8.50,0,...,3109,442,2,27,0.14,0.09,1.30,0.25,5445,844.81
2,2012,1,0.00,1.00,5,2,7,24.70,8.50,0,...,3109,442,2,27,0.14,0.09,1.30,0.25,5445,422.41
3,2012,1,0.00,1.00,5,2,7,24.70,8.50,0,...,3109,442,2,27,0.14,0.09,1.30,0.25,5445,955.00
4,2012,1,0.00,1.00,5,2,7,24.70,8.50,0,...,3109,442,2,27,0.14,0.09,1.30,0.25,5445,697.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480867,2022,12,0.86,3.10,6,3,8,29.20,8.00,1,...,6976,509,5,31,0.15,0.09,1.00,1.00,5135,370.01
480868,2022,12,0.86,3.10,6,3,8,29.20,8.00,1,...,6976,509,5,31,0.15,0.09,1.00,1.00,5135,0.00
480869,2022,12,0.86,3.10,6,3,8,29.20,8.00,1,...,6976,509,5,31,0.15,0.09,1.00,1.00,5135,447.91
480870,2022,12,0.86,3.10,6,3,8,29.20,8.00,1,...,6976,509,5,31,0.15,0.09,1.00,1.00,5135,0.00


Linear correlation

In [475]:



# Assuming you have a DataFrame called 'data' with features and the label
correlation_matrix = merged_df.corr()

# Get the correlation between features and the label
label_correlation = correlation_matrix['CrimeRate']

# Sort the correlations in descending order
sorted_correlations = label_correlation.sort_values(ascending=False)

# Set the Pandas options for display format
pd.set_option('display.float_format', lambda x: f'{x:.2f}')

# Print the correlations
print(sorted_correlations)




CrimeRate                              1.00
Tradeandcatering%                      0.33
Percentuninhabited(%)                  0.12
y65-%                                  0.11
Averagepeopleperhousehold              0.03
SelfContract                           0.02
AverageWOZ-valueofhouses(x1000euro)    0.02
NetLaborParticipation                  0.01
Rain Days                              0.00
Carsperhousehold                       0.00
Month                                 -0.00
Rainfall(mm)                          -0.00
NightC                                -0.01
DayC                                  -0.01
Daylight hours                        -0.01
FlexibleContracts                     -0.01
NuisanceReportsCriminal               -0.02
SlowResponseTimePenalty               -0.03
y0-15%                                -0.04
ResponseTimeScore                     -0.04
Year                                  -0.04
y15-25%                               -0.08
Rentalproperies(%)              

In [476]:
#better matrix
features = merged_df

# Calculate the correlation matrix
corr_matrix = features.corr()

columns = correlation_matrix.columns
n = len(columns)

# Create an empty DataFrame to store the non-zero correlations
non_zero_df = pd.DataFrame(columns=['Variable 1', 'Variable 2', 'Correlation'])

# Iterate over the upper triangular part of the correlation matrix
for i in range(n):
    for j in range(i+1, n):
        variable1 = columns[i]
        variable2 = columns[j]
        correlation = correlation_matrix.iloc[i, j]
        
        # Append the non-zero correlations to the DataFrame
        if correlation != 0:
            non_zero_df = non_zero_df.append({'Variable 1': variable1, 'Variable 2': variable2, 'Correlation': correlation}, ignore_index=True)

# Sort the DataFrame by correlation in descending order
non_zero_df = non_zero_df.sort_values('Correlation', ascending=False)
non_zero_df

C:\Users\meeh\AppData\Local\Temp\ipykernel_21556\1749035080.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  non_zero_df = non_zero_df.append({'Variable 1': variable1, 'Variable 2': variable2, 'Correlation': correlation}, ignore_index=True)
C:\Users\meeh\AppData\Local\Temp\ipykernel_21556\1749035080.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  non_zero_df = non_zero_df.append({'Variable 1': variable1, 'Variable 2': variable2, 'Correlation': correlation}, ignore_index=True)
C:\Users\meeh\AppData\Local\Temp\ipykernel_21556\1749035080.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  non_zero_df = non_zero_df.append({'Variable 1': variable1, 'Variable 2': variable2, 'Correlation': correlation}, ignore_index=True

,Variable 1,Variable 2,Correlation
118,DayC,NightC,0.97
374,PopulationEduMedium,Inhabitants,0.97
389,PopulationEduHigh,Inhabitants,0.92
343,PopulationEduLow,PopulationEduMedium,0.92
358,PopulationEduLow,Inhabitants,0.89
...,...,...,...
477,Rentalproperies(%),Carsperhousehold,-0.66
432,Peoplewithmigrationbackgroud%,AverageWOZ-valueofhouses(x1000euro),-0.66
461,AverageWOZ-valueofhouses(x1000euro),Rentalproperies(%),-0.76
314,FlexibleContracts,Averagepeopleperhousehold,-0.80


Spearman correlation

In [477]:

features = merged_df.drop('CrimeRate', axis=1)
label = merged_df['CrimeRate']

# Compute Spearman's rank-order correlation for each feature
correlation_results = []
for column in features.columns:
    correlation, p_value = spearmanr(features[column], label)
    correlation_results.append({'Feature': column, 'Correlation': correlation, 'p-value': p_value})

# Create a new dataframe to store the correlation results
correlation_df = pd.DataFrame(correlation_results)

# Sort the dataframe by absolute correlation values
correlation_df['Absolute Correlation'] = correlation_df['Correlation'].abs()
correlation_df = correlation_df.sort_values('Absolute Correlation', ascending=False)

# Print the correlation results
print(correlation_df)

                                Feature  Correlation  p-value  \
30                          Inhabitants        -0.70     0.00   
15                  PopulationEduMedium        -0.69     0.00   
16                    PopulationEduHigh        -0.67     0.00   
14                     PopulationEduLow        -0.67     0.00   
22                 Populationdensitykm2        -0.47     0.00   
29                        UrbanityLevel        -0.46     0.00   
27         Culture/recreationproperies%        -0.40     0.00   
10           NuisanceReportsNonCriminal        -0.37     0.00   
23  AverageWOZ-valueofhouses(x1000euro)         0.30     0.00   
13                         SelfContract         0.30     0.00   
25                   Rentalproperies(%)        -0.26     0.00   
20        Peoplewithmigrationbackgroud%        -0.25     0.00   
9               NuisanceReportsCriminal        -0.24     0.00   
18                              y15-25%        -0.20     0.00   
26                    Tra

In [478]:
#Model creation
if True:
    # Reset the index
    merged_df.reset_index(inplace=True)

    # Split into features and target variable
    X = merged_df.drop('CrimeRate', axis=1)
    y = merged_df['CrimeRate']

    # Remove the index from the features
    X.drop(['Year'], axis=1, inplace=True)

    # Split into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print(X_train.shape)

    # Build the Keras model
    model = Sequential()
    model.add(Dense(60, activation='tanh', input_shape=(31,)))
    model.add(Dense(12, activation='tanh'))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1))

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

    # Train the model
    history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

    # Predict on the test data
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    print("Mean Squared Error:", mse)


(384697, 31)
Epoch 1/100
12022/12022 [==============================] - 10s 802us/step - loss: 199148960.0000 - val_loss: 185251232.0000
Epoch 2/100
12022/12022 [==============================] - 10s 799us/step - loss: 188937648.0000 - val_loss: 180113680.0000
Epoch 3/100
12022/12022 [==============================] - 9s 778us/step - loss: 187090480.0000 - val_loss: 179968704.0000
Epoch 4/100
12022/12022 [==============================] - 10s 792us/step - loss: 187045632.0000 - val_loss: 179969072.0000
Epoch 5/100
12022/12022 [==============================] - 10s 795us/step - loss: 187047504.0000 - val_loss: 179969520.0000
Epoch 6/100
12022/12022 [==============================] - 10s 812us/step - loss: 187012496.0000 - val_loss: 179974240.0000
Epoch 7/100
12022/12022 [==============================] - 10s 797us/step - loss: 187041776.0000 - val_loss: 179970512.0000
Epoch 8/100
12022/12022 [==============================] - 9s 777us/step - loss: 187040768.0000 - val_loss: 179964592.00

KeyboardInterrupt: 